# 시험환경에 나와있는 작업형1 공식예제
백화점 고객의 1년 간 구매 데이터
- y_train.csv : 학습용 고객 성별 데이터 (0:여자, 1:남자)
- X_train.csv, X_test.csv : 학습, 평가용 고객의 상품구매 속성
- train으로 성별예측 모형을 만든 후, test 데이터에 적용하여 고객의 성별 예측값(남자일 확률)을 csv 결과 파일로 제출
- 평가지표 : roc_auc 
</br>

아래와 같은 형태의 csv 결과 파일 생성
```
custid, gender
3500, 0.267
3501, 0.578
````


- 시험환경에서 작업한 내용

## 1) 하이퍼 파라미터 튜닝 미적용
- 범주형 칼럼인 주구매상품의 nunique 값이 train, test에서 다름
- 따라서 원핫인코딩 시 에러 남 -> 이럴 경우 라벨인코딩
- 게다가 unique값이 20, 40개가 넘으니까 원핫인코딩 시 칼럼이 굉장히 늘어남
```
       주구매상품 주구매지점
count   3500  3500
unique    42    24
top       기타  본  점
freq     595  1077 

       주구매상품 주구매지점
count   2482  2482
unique    41    24
top       기타  본  점
freq     465   726 

KeyError: "['주구매상품_소형가전'] not in index"
```

In [ ]:
### 문제 파악
### 성별(gender) 예측, roc_auc 평가지표

# 0. 데이터 로드
import pandas as pd
X_test = pd.read_csv("data/X_test.csv")
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")


print(X_train.head(), "\n")
print(X_test.head(), "\n")
print(y_train.head())



# 1. EDA
# 1-1) train EDA
print(X_train.info())
print(X_train.isnull().sum()) # 결측치 : 환불금액 2295
print(X_train.describe(), "\n")

# 1-2) test EDA
print(X_test.info())
print(X_test.isnull().sum()) # 결측치 : 환불금액 1611
print(X_test.describe(), "\n")

print(X_train.describe(include= "O"), "\n") # train, test에서 nunique값이 달라서 원핫인코딩 시 에러남
print(X_test.describe(include= "O"), "\n")


# 2. 데이터 전처리
# 2-1) 결측치 대체, ID 같은 불필요 칼럼 제거
X_train["환불금액"] = X_train["환불금액"].fillna(0)
X_test["환불금액"] = X_test["환불금액"].fillna(0)

print(X_train.isnull().sum(), "\n")
print(X_test.isnull().sum())

# 불필요 칼럼 제거 확인
X_train = X_train.drop("cust_id", axis=1)
print(X_train.head(1), "\n")

test_cust_id = X_test.pop("cust_id")
print(X_test.head(1), "\n")

y = y_train["gender"] # 아예 y 객체에 gender 값만 할당
print(y.head(), "\n")

# 2-2) 스케일링
print(X_train.info())

con_cols = X_train.select_dtypes(exclude="object").copy().columns  # 수치형 칼럼명 선택
# con_df = X_train.select_dtypes(include=['float64', 'int64']).copy().columns # 수치형 칼럼명 가져오기
print(con_cols)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train[con_cols] = scaler.fit_transform(X_train[con_cols])
X_test[con_cols] = scaler.transform(X_test[con_cols])

print(X_train.head(), "\n")
print(X_test.head(), "\n")

# 2-3) 인코딩
# 범주형 변수 안에 있는 nunique 값이 많은 편이니 라벨인코딩 해주자
cat_cols = X_train.select_dtypes(include="object").copy().columns # 범주형 변수 선택
# print(cat_cols)

from sklearn.preprocessing import LabelEncoder
for col in cat_cols :
	le = LabelEncoder()
	X_train[col] = le.fit_transform(X_train[col])
	X_test[col] = le.transform(X_test[col])
	
print(X_train.head()) # 인코딩 적용 확인



# 3. 검증 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y, random_state=1, stratify=y)
print(X_train.shape, X_val.shape)



# 4. 모델링
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import auc,roc_auc_score

# 4-1) 랜덤포레스트 분류
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=2023)
rf.fit(X_train, y_train)
rf_pred_proba = rf.predict_proba(X_val) 
print("rf 정확도 : ", rf.score(X_val, y_val)) # 0.648

rf_auc = roc_auc_score(y_val, rf_pred_proba[:, 1])
print("rf_auc : ", rf_auc) # 0.653

# 4-2) lgbm 분류
lgbm = LGBMClassifier(n_estimators=100, max_depth=5, random_state=2023)
lgbm.fit(X_train, y_train)
lgbm_pred_proba = lgbm.predict_proba(X_val) 
print("lgbm 정확도 : ", lgbm.score(X_val, y_val))

lgbm_auc = roc_auc_score(y_val, lgbm_pred_proba[:,1])
print("lgbm_auc : ", lgbm_auc) # 0.64

# 최종모델은 lgbm
pred = lgbm.predict_proba(X_test)[:, 1] # test 셋에서 남자일 예측 '확률'



# 5) 저장
output = pd.DataFrame({'cust_id': test_cust_id, 'gender': pred})
print(output.head())

output.to_csv("수험번호.csv", index=False)

# result = pd.read_csv("수험번호.csv")
# print(result.head())

## 2) 하이퍼 파라미터 튜닝 적용 : 그리드 서치
```
==========
model : RandomForestClassifier(random_state=1)
params : {'max_depth': 3, 'n_estimators': 200}
score : 0.6684345665421588
==========
model : LGBMClassifier(random_state=1)
params : {'max_depth': 2, 'n_estimators': 100}
score : 0.6811735359034122
```

In [ ]:
### 문제 파악
### 성별(gender) 예측, roc_auc 평가지표

# 0. 데이터 로드
import pandas as pd
X_test = pd.read_csv("data/X_test.csv")
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")


print(X_train.head(), "\n")
print(X_test.head(), "\n")
print(y_train.head())



# 1. EDA
# 1-1) train EDA
print(X_train.info())
print(X_train.isnull().sum()) # 결측치 : 환불금액 2295
print(X_train.describe(), "\n")

# 1-2) test EDA
print(X_test.info())
print(X_test.isnull().sum()) # 결측치 : 환불금액 1611
print(X_test.describe(), "\n")

print(X_train.describe(include= "O"), "\n") # train, test에서 nunique값이 달라서 원핫인코딩 시 에러남
print(X_test.describe(include= "O"), "\n")


# 2. 데이터 전처리
# 2-1) 결측치 대체, ID 같은 불필요 칼럼 제거
X_train["환불금액"] = X_train["환불금액"].fillna(0)
X_test["환불금액"] = X_test["환불금액"].fillna(0)

print(X_train.isnull().sum(), "\n")
print(X_test.isnull().sum())

# 불필요 칼럼 제거 확인
X_train = X_train.drop("cust_id", axis=1)
print(X_train.head(1), "\n")

test_cust_id = X_test.pop("cust_id")
print(X_test.head(1), "\n")

y = y_train["gender"] # 아예 y 객체에 gender 값만 할당
print(y.head(), "\n")

# 2-2) 스케일링
print(X_train.info())

con_cols = X_train.select_dtypes(exclude="object").copy().columns  # 수치형 칼럼명 선택
# con_df = X_train.select_dtypes(include=['float64', 'int64']).copy().columns # 수치형 칼럼명 가져오기
print(con_cols)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train[con_cols] = scaler.fit_transform(X_train[con_cols])
X_test[con_cols] = scaler.transform(X_test[con_cols])

print(X_train.head(), "\n")
print(X_test.head(), "\n")

# 2-3) 인코딩
# 범주형 변수 안에 있는 nunique 값이 많은 편이니 라벨인코딩 해주자
cat_cols = X_train.select_dtypes(include="object").copy().columns # 범주형 변수 선택
# print(cat_cols)

from sklearn.preprocessing import LabelEncoder
for col in cat_cols :
	le = LabelEncoder()
	X_train[col] = le.fit_transform(X_train[col])
	X_test[col] = le.transform(X_test[col])
	
print(X_train.head()) # 인코딩 적용 확인



# 3. 검증 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y, random_state=1, stratify=y)
print(X_train.shape, X_val.shape)


# 4. 모델링
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import auc,roc_auc_score

# rf = RandomForestClassifier(random_state=1)
# lgbm = LGBMClassifier(random_state=1)

# models = [rf, lgbm]

# # Hyperparameter tuning
# from sklearn.model_selection import GridSearchCV

# params = {"n_estimators":[100,200,300], "max_depth":[1,2,3]}

# # best_models = []

# for model in models:
#     gs = GridSearchCV(model, param_grid=params, cv = 5, scoring="roc_auc", n_jobs=4)
#     gs.fit(X_train, y_train)

#     print(f"="*10)
#     print(f"model : {model}")
#     print(f"params : {gs.best_params_}")
#     print(f"score : {gs.best_score_}")
		
		

# 최종 모델 : lgbm
lgbm = LGBMClassifier(random_state=1, max_depth=2, n_estimators=100)
lgbm.fit(X_train, y_train)
lgbm_pred_proba = lgbm.predict_proba(X_val)[:, 1]

lgbm_roc = roc_auc_score(y_val, lgbm_pred_proba)
print(lgbm_roc) # 0.65
# train roc_auc_score : 0.68

pred = lgbm.predict_proba(X_test)[:, 1]

submit = pd.DataFrame({"custid" : test_cust_id, "gender" : pred})
submit.to_csv("수험번호.csv", index=False)
check = pd.read_csv("수험번호.csv")
print(check.head())